In [1]:
import sys 
import os 
print(sys.path)

['C:\\Users\\camin\\PhD_DB\\CV2020-project_iMaterialist', 'C:\\Users\\camin\\anaconda3\\envs\\pytorch_env\\python37.zip', 'C:\\Users\\camin\\anaconda3\\envs\\pytorch_env\\DLLs', 'C:\\Users\\camin\\anaconda3\\envs\\pytorch_env\\lib', 'C:\\Users\\camin\\anaconda3\\envs\\pytorch_env', '', 'C:\\Users\\camin\\anaconda3\\envs\\pytorch_env\\lib\\site-packages', 'C:\\Users\\camin\\anaconda3\\envs\\pytorch_env\\lib\\site-packages\\win32', 'C:\\Users\\camin\\anaconda3\\envs\\pytorch_env\\lib\\site-packages\\win32\\lib', 'C:\\Users\\camin\\anaconda3\\envs\\pytorch_env\\lib\\site-packages\\Pythonwin', 'C:\\Users\\camin\\anaconda3\\envs\\pytorch_env\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\camin\\.ipython']


In [2]:
import numpy as np 
import json 
import pandas as pd 
from pathlib import Path
from tqdm.notebook import tqdm

In [3]:
import torch 
import torch.nn as nn 
import torch.optim as optim 

import torchvision
from torchvision import datasets, transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torch.optim.lr_scheduler import MultiStepLR

In [4]:
from DataLoader import Fashion2020dataset

In [5]:
data_dir = Path("../imaterialist-fashion-2020-fgvc7")
df_csv   = Path("../imaterialist-fashion-2020-fgvc7/train.csv")

Dataloader = Fashion2020dataset(root= data_dir, transforms=None, csv_path=df_csv)

In [6]:
Dataloader.__len__()

45623

In [7]:
len(Dataloader.img_lists)

45623

In [8]:
train_idx = [] 
val_idx = [] 

for i, img_file in enumerate(Dataloader.img_lists):
    if i % 10 == 0 :
        val_idx.append(i)
    else:
        train_idx.append(i)

In [9]:
len(val_idx)

4563

In [10]:
len(train_idx)

41060

In [11]:
test_idx = np.random.choice(41060, size=4562, replace=False, p=None)
test_idx.sort()

In [24]:
test_idx

array([   40,    58,    74, ..., 41016, 41019, 41050])

In [13]:
new_train = set(np.arange(41060)) - set(test_idx)
new_train = np.array(list(new_train))

In [22]:
len(new_train)

36498

## Read json 

In [15]:
trainFix_json = os.path.join(data_dir, "trainFix.json" )

with open(trainFix_json, 'r') as file:
    coco_json = json.load(file)

In [16]:
type(coco_json)

dict

In [17]:
coco_json.keys()

dict_keys(['images', 'annotations', 'categories'])

In [18]:
train_json = {}
test_json  = {} 

In [25]:
coco_json['images'][0]

{'file_name': '0000fe7c9191fba733c8a69cfaf962b7.jpg',
 'height': 2448,
 'width': 2448,
 'id': 0}

In [19]:
train_json['categories'] = coco_json['categories']
test_json['categories'] = coco_json['categories']

True

In [37]:
len(coco_json['annotations'])

216485

In [43]:
train_json['images'] = [] 
train_json['annotations']  = [] 

for i, ID in tqdm(enumerate(new_train), desc='Processing'):
    train_json['images'].append(coco_json['images'][ID])
    
    for idx, val in enumerate(coco_json['annotations']): 
        if coco_json['annotations'][idx]['image_id'] == ID:
            train_json['annotations'].append(coco_json['annotations'][idx])
    
    
    

In [44]:
save_path = os.path.join(data_dir, "trainFix2.json" )

with open(save_path, 'w') as f:
    json.dump(train_json, f)